# Metadata

```
Course:   DS 5100
Module:   06 Pandas
Topic:    Introducing Pandas II
Author:   R.C. Alvarado (adapted)
Date:     28 June 2022
```

# Set Up

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')

In [3]:
iris.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [4]:
import sys
sys.getsizeof(iris)

14744

# Apply Lambda Functions with `.apply()`

Apply a transformation to each record. Uses a `lambda` function.

The `apply()` method should be used after you have established that you can't use a vectorized function.

In [5]:
iris['sepal_len_sq'] = iris.sepal_length.apply(lambda x: x**2)
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq
0,5.1,3.5,1.4,0.2,setosa,26.01
1,4.9,3.0,1.4,0.2,setosa,24.01
2,4.7,3.2,1.3,0.2,setosa,22.09
3,4.6,3.1,1.5,0.2,setosa,21.16
4,5.0,3.6,1.4,0.2,setosa,25.00


Transformation involving multiple columns. Uses `axis=1` to access columns.  
Compute average of `sepal_length`, `sepal_width`:

In [6]:
iris['sepal_len_wid_avg'] = iris[['sepal_length','sepal_width']].apply(lambda x: (x.sepal_length+x.sepal_width)/2, axis=1)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq,sepal_len_wid_avg
0,5.1,3.5,1.4,0.2,setosa,26.01,4.30
1,4.9,3.0,1.4,0.2,setosa,24.01,3.95
2,4.7,3.2,1.3,0.2,setosa,22.09,3.95
3,4.6,3.1,1.5,0.2,setosa,21.16,3.85
4,5.0,3.6,1.4,0.2,setosa,25.00,4.30


**Vectorized Version**

In [7]:
%time iris.sepal_length**2

CPU times: user 5.57 ms, sys: 5.25 ms, total: 10.8 ms
Wall time: 30.1 ms


0      26.01
1      24.01
2      22.09
3      21.16
4      25.00
       ...  
145    44.89
146    39.69
147    42.25
148    38.44
149    34.81
Name: sepal_length, Length: 150, dtype: float64

Compare to `.apply()`

In [8]:
%time iris.sepal_length.apply(lambda x: x**2)

CPU times: user 440 µs, sys: 28 µs, total: 468 µs
Wall time: 456 µs


0      26.01
1      24.01
2      22.09
3      21.16
4      25.00
       ...  
145    44.89
146    39.69
147    42.25
148    38.44
149    34.81
Name: sepal_length, Length: 150, dtype: float64

# Aggregation

Involves one or more of:

- splitting the data into groups
- applying a function to each group
- combining results

## `.groupby()`

Compute mean of each column, grouped (separately) by species

In [9]:
iris.groupby("species").mean()

,sepal_length,sepal_width,petal_length,petal_width,sepal_len_sq,sepal_len_wid_avg
species,,,,,,
setosa,5.006,3.428,1.462,0.246,25.1818,4.217
versicolor,5.936,2.770,4.260,1.326,35.4972,4.353
virginica,6.588,2.974,5.552,2.026,43.7980,4.781


## `pd.pivot_table()`

Apply a function `aggfunc` to selected values grouped by columns

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

Compute mean sepal length for each species:

In [10]:
pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = np.mean)

species,setosa,versicolor,virginica
sepal_length,5.006,5.936,6.588


# Stacking and Unstacking

Similar to pivoting, but requires -- and takes advantage of -- indexes.

In [11]:
iris_w_idx = iris.copy() 

# Give the original index a name
iris_w_idx.index.name = 'obs_id'

# Create a multi-index, using `species` as part of the key.
iris_w_idx = iris_w_idx.reset_index().set_index(['species','obs_id'])

In [12]:
iris_w_idx

sepal_length  sepal_width  petal_length  petal_width  \
species   obs_id                                                         
setosa    0                5.1          3.5           1.4          0.2   
          1                4.9          3.0           1.4          0.2   
          2                4.7          3.2           1.3          0.2   
          3                4.6          3.1           1.5          0.2   
          4                5.0          3.6           1.4          0.2   
...                        ...          ...           ...          ...   
virginica 145              6.7          3.0           5.2          2.3   
          146              6.3          2.5           5.0          1.9   
          147              6.5          3.0           5.2          2.0   
          148              6.2          3.4           5.4          2.3   
          149              5.9          3.0           5.1          1.8   

                  sepal_len_sq  sepal_len_wid_avg  
species   obs_id                                   
setosa    0              26.01               4.30  
          1              24.01               3.95  
          2              22.09               3.95  
          3              21.16               3.85  
          4              25.00               4.30  
...                        ...                ...  
virginica 145            44.89               4.85  
          146            39.69               4.40  
          147            42.25               4.75  
          148            38.44               4.80  
          149            34.81               4.45  

[150 rows x 6 columns]

## `.unstack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html)

In [16]:
iris_wide = iris_w_idx.sepal_length.unstack(fill_value=0).T

In [17]:
iris_wide

species,setosa,versicolor,virginica
obs_id,,,
0,5.1,0.0,0.0
1,4.9,0.0,0.0
2,4.7,0.0,0.0
3,4.6,0.0,0.0
4,5.0,0.0,0.0
...,...,...,...
145,0.0,0.0,6.7
146,0.0,0.0,6.3
147,0.0,0.0,6.5


In [18]:
iris_wide.mean()

species
setosa        1.668667
versicolor    1.978667
virginica     2.196000
dtype: float64

## `.stack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html)

In [15]:
iris_wide.T.stack().to_frame('sepal_length')

sepal_length
species   obs_id              
setosa    0                5.1
          1                4.9
          2                4.7
          3                4.6
          4                5.0
...                        ...
virginica 145              6.7
          146              6.3
          147              6.5
          148              6.2
          149              5.9

[450 rows x 1 columns]

# Combining DataFrames

## `pd.concat()`  

Concatenate pandas objects along an axis.

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

Create two dfs and vertically stack them

In [16]:
df1 = pd.DataFrame(np.random.randn(3, 4))
df2 = pd.DataFrame(np.random.randn(3, 4))

In [17]:
df1

,0,1,2,3
0,-0.529316,0.439009,-1.192926,0.487157
1,0.719213,-2.363081,0.823607,0.523713
2,0.124701,-1.810517,-0.970098,-1.178179


In [18]:
df2

,0,1,2,3
0,0.970977,2.528656,1.975030,-1.298264
1,1.600237,-0.744094,-1.064893,1.385589
2,-1.175151,-0.061801,1.026631,0.516147


**Concat rows**

In [19]:
df3 = pd.concat([df1, df2], axis=0)

In [20]:
df3

,0,1,2,3
0,-0.529316,0.439009,-1.192926,0.487157
1,0.719213,-2.363081,0.823607,0.523713
2,0.124701,-1.810517,-0.970098,-1.178179
0,0.970977,2.528656,1.975030,-1.298264
1,1.600237,-0.744094,-1.064893,1.385589
2,-1.175151,-0.061801,1.026631,0.516147


**Concat columns**

This assumes that the indexes represent IDs of specific things or events.

In [21]:
df4 = pd.concat([df1, df2], axis=1, keys=['foo', 'bar'])

In [22]:
df4

foo                                     bar                      \
          0         1         2         3         0         1         2   
0 -0.529316  0.439009 -1.192926  0.487157  0.970977  2.528656  1.975030   
1  0.719213 -2.363081  0.823607  0.523713  1.600237 -0.744094 -1.064893   
2  0.124701 -1.810517 -0.970098 -1.178179 -1.175151 -0.061801  1.026631   

             
          3  
0 -1.298264  
1  1.385589  
2  0.516147

In [23]:
df4.foo

,0,1,2,3
0,-0.529316,0.439009,-1.192926,0.487157
1,0.719213,-2.363081,0.823607,0.523713
2,0.124701,-1.810517,-0.970098,-1.178179


## `.merge()`

SQL-style joining of tables (DataFrames) -- although Pandas has a `.join()` method, too.

Important parameters include:

- `how` : type of merge {'left', 'right', 'outer', 'inner', 'cross'}, default ‘inner’
- `on`  : names to join on
        
[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Create two tables, `left` and `right`. Then right join them on `key`.  
Right join means include all records from table on right.  
The `key` is used for matching up the records.

In [24]:
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  lval
0  jamie    15
1   bill    22

---right
     key  rval
0  jamie     4
1   bill     5
2  asher     8

---joined
     key  lval  rval
0  jamie  15.0     4
1   bill  22.0     5
2  asher   NaN     8


Notice the NaN inserted into the record with key=asher, since the left table didn't contain the key.

**Matching column names**  
In this next example, the value columns have the same name: *val*.  Notice what happens to the column names.

In [25]:
left = pd.DataFrame({"key": ["jamie", "bill"], "val": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "val": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  val
0  jamie   15
1   bill   22

---right
     key  val
0  jamie    4
1   bill    5
2  asher    8

---joined
     key  val_x  val_y
0  jamie   15.0      4
1   bill   22.0      5
2  asher    NaN      8


## `.join()`

An SQL-like joiner, but this one takes advantage of indexes.

Give our dataframes indexes and distinctive columns names.

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)

In [26]:
left2 = left.set_index('key').rename(columns={'val':'val_1'})
right2 = right.set_index('key').rename(columns={'val':'val_2'})

In [27]:
left2

,val_1
key,
jamie,15
bill,22


In [28]:
right2

,val_2
key,
jamie,4
bill,5
asher,8


In [29]:
right2.join(left2) # Defaults to 'inner'

,val_2,val_1
key,,
jamie,4,15.0
bill,5,22.0
asher,8,NaN


In [30]:
right2.join(left2, how='left')

,val_2,val_1
key,,
jamie,4,15.0
bill,5,22.0
asher,8,NaN


## Summary

* Use **join** if you have shared indexes
* Use **merge** if you do not have shared indexes
* Use **concat** to combine based on shared indexes or columns

# Reshape with `.reshape()`

Changes the object's shape

We illustrate creating pandas Series, extracting array of length 6, and reshaping to 3x2 array.

In [31]:
# create a series 
ser = pd.Series([1, 1, 2, 3, 5, 8]) 

# extract values 
vals = ser.values 

print('orig data:', vals)
print('orig type:', type(vals))
print('orig shape:', vals.shape)

# reshaping series
reshaped_vals = vals.reshape((3, 2)) 

print('\n reshaped vals:')
print(reshaped_vals)
print('\n new type:', type(reshaped_vals))
print('new shape:', reshaped_vals.shape)

orig data: [1 1 2 3 5 8]
orig type: <class 'numpy.ndarray'>
orig shape: (6,)

 reshaped vals:
[[1 1]
 [2 3]
 [5 8]]

 new type: <class 'numpy.ndarray'>
new shape: (3, 2)


Including -1 as one of the dimensions tells numpy: infer this dimension from the data and the other dimensions.

Example: enforce 3 columns:

In [32]:
vals.reshape(-1,3)

array([[1, 1, 2],
       [3, 5, 8]])

Enforce 3 rows:

In [33]:
vals.reshape(3,-1)

array([[1, 1],
       [2, 3],
       [5, 8]])

**IMPORTANT NOTE**  

Notice the shape of original array: `(6,)`  
This is a vector with one dimension, and is different from two-dimensional `(6,1)` array

# Categoricals

Categorical data takes discrete values where computation on the values does not make sense.  
Zip code is a typical example.

To include categoricals in models, they must be converted to numeric.  

## `get_dummies()`
Dummy code categorical data

Important parameters: 

- `prefix`    : append prefix to column names (a good idea for later use)
- `drop_first`: remove first level, as only `k-1` variables needed to represent `k` levels

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

In [36]:
cats = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese']})

In [37]:
cats

,breed
0,persian
1,persian
2,siamese
3,himalayan
4,burmese


In [38]:
dummy_cats = pd.get_dummies(cats.breed, drop_first=True, prefix='breed')

In [39]:
dummy_cats

,breed_himalayan,breed_persian,breed_siamese
0,0,1,0
1,0,1,0
2,0,0,1
3,1,0,0
4,0,0,0


Notice `burmese` was dropped (first level by alphabet) since it can be inferred.